In [1]:
import sys
import os
sys.path.append('/g/data/jk72/ll6859/access_aero_eval/')
from aercode import *
import matplotlib 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb
import cartopy.crs as ccrs
import seaborn as sns
import scipy.special
from datetime import datetime

import dask
import warnings

In [2]:
from dask.distributed import Client
client = Client()  # set up local cluster
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 4,Total memory: 18.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35509,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 18.00 GiB
Comm: tcp://127.0.0.1:45159,Total threads: 1
Dashboard: /proxy/44597/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:42229,


In [3]:
dask.config.set(**{'array.slicing.split_large_chunks': True});

In [4]:
plt.style.use('default')
warnings.filterwarnings('ignore')

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [6]:
output_path = '/g/data/jk72/ll6859/access_aero_eval/output/'

# RVInvestigator

In [7]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/access_aero_eval/RVInvestigator/'
aer = pd.read_csv(f_path+'rvigaw_cn10_2016to2019_L2.csv', index_col='datetime')
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time']
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn10_avg','cn10_std','latitude','longitude']].dropna()
aer.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)

met = pd.read_csv(f_path+'RVI-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']>60]=np.nan
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2020_05_29d_RVI_1420_Radon.xlsx', sheet_name='RVI1420', index_col='DateTime')
rdn['time'] = pd.DatetimeIndex(rdn.index)
rdn.index = rdn['time']
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]','RnSTP']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std', 'RnSTP': 'radon_stp'}, inplace=True)

In [8]:
rvi_obs = pd.concat([aer,met,rdn], axis=1, join='inner').reset_index()

In [9]:
rvi_obs['Source'] = 'RVI'
rvi_obs = rvi_obs.set_index(['time','Source'])

# Cold Water Trial

In [10]:
# f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/ColdWaterTrial/'

# df = pd.read_csv(f_path+'CWT_ProcessedAerosolData_1min.csv', delimiter=',', index_col='date')
# df['time'] = pd.DatetimeIndex(df.index)
# df.index = df['time'].astype('datetime64[ns]')
# df = df[['lat','lon','AirTemp_port','AirTemp_stbd','atmPress','WindSpdTru','Radon_hrly','CN_mean']]
# df = df.dropna()
# df = df.resample('1D', kind='time').mean()

# Reef 2 Rainforest

In [11]:
f_path = '/g/data/jk72/ll6859/access_aero_eval/Reef2Rain/'

df = pd.read_csv(f_path+'R2R_RV_aerosol.csv', index_col='date')
df['time'] = pd.DatetimeIndex(df.index)
df.index = df['time'].astype('datetime64[ns]')
df = df.resample('1D', kind='time').mean()
df = df.dropna()
df['air_temp'] = (df['AirTemp_port']+df['AirTemp_stbd'])/2
df['wind_spd'] = (df['WindSpdTru_port']+df['WindSpdTru_stbd'])/2
df = df[['lat','lon','air_temp','atmPress','WindDir', 'wind_spd', 'Radon_hrly','CN3_mean','CN10_mean','CCN_mean']]
df.rename(columns={'atmPress': 'air_pressure', 
                   'WindDir': 'wind_dir', 
                   'Radon_hrly': 'radon', 
                   'CN3_mean': 'cn3_avg', 
                   'CN10_mean': 'cn10_avg', 
                   'CCN_mean': 'ccn_avg'}, inplace=True)

In [12]:
r2r_obs = df.reset_index()

In [13]:
r2r_obs['Source'] = 'Reef2Rain'
r2r_obs = r2r_obs.set_index(['time','Source'])

# Ice 2 Equator

In [14]:
f_path = '/g/data/jk72/ll6859/access_aero_eval/Ice2Equator/'

df = pd.read_csv(f_path+'I2E_ProcessedAerosolData_1min.csv', index_col='date')
df['time'] = pd.DatetimeIndex(df.index)
df.index = df['time'].astype('datetime64[ns]')
df = df.resample('1D', kind='time').mean()
df = df.dropna()
df['air_temp'] = (df['AirTemp_port']+df['AirTemp_stbd'])/2
df['wind_spd'] = (df['WindSpdTru_port']+df['WindSpdTru_stbd'])/2
df = df[['lat','lon','air_temp','atmPress','wind_spd','WindDirRel_vmean','Radon_hrly','CN_mean','CN_std']]
df.rename(columns={'atmPress': 'air_pressure',
                   'WindDirRel_vmean':'wind_dir',
                   'Radon_hrly': 'radon', 
                   'CN_mean': 'cn3_avg', 
                   'CN_std': 'cn3_std'}, inplace=True)

In [15]:
i2e_obs = df.reset_index()

In [16]:
i2e_obs['Source'] = 'Ice2Equ'
i2e_obs = i2e_obs.set_index(['time','Source'])

# CAPRICORN I

In [17]:
f_path = '/g/data/jk72/ll6859/access_aero_eval/Capricorn/'

aer = pd.read_hdf(f_path+'aerosol_1sec.h5')
aer.index.name = 'time'
aer = aer.resample('1D', kind='time').mean()
aer.rename(columns={'ccn_0.5504': 'ccn_avg',
                   'ccn_sigma': 'ccn_std', 
                   'cn3': 'cn3_avg', 
                   'cn3_sigma': 'cn3_std'}, inplace=True)

met = pd.read_csv(f_path+'in2016_v02uwy10sec.csv', parse_dates=[['date', 'time']], index_col='date_time')
met = met.resample('1D').mean()
met['air_temp'] = (met['portAirTemp(degC)']+met['stbdAirTemp(degC)'])/2
met = met[['latitude(degree_north)','longitude(degree_east)','air_temp','atmPressure(mbar)','rain(mm)']]
met.rename(columns = {"latitude(degree_north)": "lat",
                      "longitude(degree_east)": "lon",
                      "atmPressure(mbar)": "air_pressure",
                      "rain(mm)": "total_rain"}, inplace=True)
met.index.name = 'time'   

In [18]:
cap1_obs = pd.concat([aer,met], axis=1, join='inner').reset_index()

In [19]:
cap1_obs['Source'] = 'Capricorn1'
cap1_obs = cap1_obs.set_index(['time','Source'])

# CAPRICORN II

In [20]:
f_path = '/g/data/jk72/ll6859/access_aero_eval/Capricorn/'

aer = pd.read_csv(f_path+'capricorn2_cpc3772_L2_1H.csv', index_col=0)
aer.index.name = 'time'
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time'].astype('datetime64[ns]')
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn_avg', 'cn_std']]
aer.rename(columns={'cn_avg': 'cn10_avg',
                   'cn_std': 'cn10_std'}, inplace=True)

met = pd.read_csv(f_path+'in2018_v01uwy10sec.csv', parse_dates=[['date', 'time']], index_col='date_time')
for v in met.select_dtypes('object'):
    met[v] = pd.to_numeric(met[v], errors='coerce')
met.index.name = 'time'
met = met.resample('1D').mean()
met['air_temp'] = (met['portAirTemp(degC)']+met['stbdAirTemp(degC)'])/2
met['total_rain'] = (met['portRain(mm)']+met['stbdRain(mm)'])/2
met['wind_spd'] = (met['portTrueWindSpeed(knot)']+met['stbdTrueWindSpeed(knot)'])/2*1.944
met['wind_dir'] = (met['portTrueWindDir(degree)']+met['stbdTrueWindDir(degree)'])/2
met = met[['latitude(degree_north)','longitude(degree_east)','air_temp','atmPressure(mbar)','total_rain','wind_spd','wind_dir']]
met.rename(columns = {"latitude(degree_north)": "lat",
                            "longitude(degree_east)": "lon",
                            "atmPressure(mbar)": "air_pressure"}, inplace=True)

In [21]:
cap2_obs = pd.concat([aer,met], axis=1, join='inner').reset_index()

In [22]:
cap2_obs['Source'] = 'Capricorn2'
cap2_obs = cap2_obs.set_index(['time','Source'])

# MARCUS

In [23]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/access_aero_eval/AuroraAustralis/'

aer = pd.read_csv(f_path+'CN_MARCUS_1hr.csv', index_col=0)
aer.index.name = 'time'
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time']
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn_avg','cn_std','lat','lon']].dropna()
aer.rename(columns={'cn_avg': 'cn10_avg', 'cn_std': 'cn10_std'}, inplace=True)

met = pd.read_csv(f_path+'marcus-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

In [24]:
marcus_obs = pd.concat([aer,met], axis=1, join='inner').reset_index()

In [25]:
marcus_obs['Source'] = 'MARCUS'
marcus_obs = marcus_obs.set_index(['time','Source'])

# TAN1802

In [26]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/access_aero_eval/TAN1802/'
aer = xr.open_dataset(f_path+'CPC3010_TAN1802_2018_level_01.nc')
aer = aer.rename({'number_concentration_of_ambient_aerosol_particles_in_air':'cn10',
                  'number_concentration_of_ambient_aerosol_particles_in_air_flag':'cn10_flag'})
aer = aer.where(aer.cn10_flag == 1)
aer['cn10'] = aer['cn10']/1000000
aer = aer.resample(time='1D').mean()
aer = aer.rename({'cn10': 'cn10_avg'})
aer = aer.to_dataframe()

uw = keycutter(f_path+'weather.csv','Time (UTC)', 'Latitude', 'Longitude')


met = pd.read_csv(f_path+'TAN-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

In [27]:
tan1802_obs = pd.concat([aer,uw,met], axis=1, join='inner').reset_index()

In [28]:
tan1802_obs['Source'] = 'TAN1802'
tan1802_obs = tan1802_obs.set_index(['time','Source'])

# Cape Grim

In [29]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/access_aero_eval/CGO/'
aer = pd.read_excel(f_path+'cgbpaps_CN  2016-2018.xlsx', index_col='date')
aer = aer.astype(np.float64).resample('1D',kind='time').mean()
aer.index.name = 'time'
aer = aer[['cn3772_mean','cn3772_sd']]
aer.rename(columns={'cn3772_mean': 'cn10_avg', 'cn3772_sd': 'cn10_std'}, inplace=True)

met = pd.read_excel(f_path+'CG_met_2015-2018.xlsx', index_col='date')
met = met[['TempGC','Pressure','Rain','VWS10m','VWD10m']]
met.index.name = 'time'
met = met.resample('1D', kind='time').mean()
met.rename(columns={'TempGC': 'air_temp', 'Rain': 'total_rain', 'Pressure': 'air_pressure', 'VWD10m': 'wind_dir', 'VWS10m': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2021_08_25a_CGO_Radon_2015-2018_ForLiam.xlsx', sheet_name='2015-2018-RadonMetCO2', index_col='DateTime')
rdn.index.name = 'time'
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std'}, inplace=True)

In [30]:
cgo_obs = pd.concat([aer,met,rdn], axis=1, join='inner').reset_index()

In [31]:
cgo_obs['Source'] = 'CGO'
cgo_obs = cgo_obs.set_index(['time','Source'])

# Macquarie Island

In [32]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/access_aero_eval/MQI/'
aer = pd.read_hdf(f_path+'ACRE_CN10_1H_FINAL.h5')
aer.index.name = 'time'
aer = aer[['cn_avg','cn_std']]
aer = aer.resample('1D', kind='time').mean()
aer.rename(columns={'cn_avg': 'cn10_avg', 'cn_std': 'cn10_std'}, inplace=True)

met = []
all_files = gb.glob(f_path+'AWS_hourly/*.csv')
met = pd.concat([pd.read_csv(v) for v in all_files])
met.index = met['MIN_OBSERVATION_DATE_'].astype('datetime64[ns]')
met.index.name = 'time'
met['AVG_RAINFALL'] = met['AVG_RAINFALL'].diff()
met['AVG_RAINFALL'][met['AVG_RAINFALL']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['AVG_RAINFALL'] = met_tmp['AVG_RAINFALL'].resample('1D', kind='time').sum()
met = met[['AVG_AIR_TEMPERATURE','AVG_WIND_DIRECTION','AVG_WIND_SPEED','AVG_AIR_PRESSURE','AVG_RAINFALL']]
met.rename(columns={'AVG_AIR_TEMPERATURE': 'air_temp', 'AVG_RAINFALL': 'total_rain', 'AVG_AIR_PRESSURE': 'air_pressure', 'AVG_WIND_DIRECTION': 'wind_dir', 'AVG_WIND_SPEED': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2021_09_15a_MQI_Radon_2013-2019_ForLiamLamprey.xlsx',
                    sheet_name='Radon_MQI_2013-2019',
                    index_col='DateTime')
rdn.index.name = 'time'
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std'}, inplace=True)

In [33]:
mqi_obs = pd.concat([aer,met,rdn], axis=1, join='inner').reset_index()

In [34]:
mqi_obs['Source'] = 'MQI'
mqi_obs = mqi_obs.set_index(['time','Source'])

# Combine obs data

In [35]:
voyages = [rvi_obs,
           r2r_obs,
           i2e_obs,
           cap1_obs,
           cap2_obs,
           marcus_obs,
           tan1802_obs,
           cgo_obs,
           mqi_obs,
]

voyage_obs_ds = pd.concat(voyages).to_xarray()
voyage_obs_ds

<xarray.Dataset>
Dimensions:       (time: 1223, Source: 9)
Coordinates:
  * time          (time) datetime64[ns] 2016-01-01 2016-01-02 ... 2019-06-12
  * Source        (Source) object 'CGO' 'Capricorn1' ... 'Reef2Rain' 'TAN1802'
Data variables: (12/17)
    cn10_avg      (time, Source) float64 1.763e+03 nan nan nan ... 165.1 nan nan
    cn10_std      (time, Source) float64 74.55 nan nan nan ... nan 6.967 nan nan
    lat           (time, Source) float64 nan nan nan nan ... nan -27.79 nan nan
    lon           (time, Source) float64 nan nan nan nan ... nan 113.3 nan nan
    air_temp      (time, Source) float64 17.17 nan nan nan ... nan 17.51 nan nan
    air_pressure  (time, Source) float64 1.002e+03 nan nan ... 1.022e+03 nan nan
    ...            ...
    radon_stp     (time, Source) float64 nan nan nan nan ... nan 39.9 nan nan
    cn3_avg       (time, Source) float64 nan nan nan nan nan ... nan nan nan nan
    ccn_avg       (time, Source) float64 nan nan nan nan nan ... nan nan nan nan
    cn3_std       (time, Source) float64 nan nan nan nan nan ... nan nan nan nan
    ccn_std       (time, Source) float64 nan nan nan nan nan ... nan nan nan nan
    cn10_flag     (time, Source) float32 nan nan nan nan nan ... nan nan nan nan

In [37]:
voyage_obs_ds.load().to_netcdf(path=output_path+'voyage_obs.nc')

## Combine model data

In [38]:
# load in data
f_path = '/g/data/jk72/ll6859/access_aero_eval/'
rvi_bx = xr.open_dataset(f_path+'RVInvestigator/rvi1619_bx400_track.nc').assign_coords(Source='RVI')

r2r_bx = xr.open_dataset(f_path+'Reef2Rain/r2r16_bx400_track.nc').assign_coords(Source='Reef2Rain')

i2e_bx = xr.open_dataset(f_path+'Ice2Equator/i2e16_bx400_track.nc').assign_coords(Source='Ice2Equ')

cap1_bx = xr.open_dataset(f_path+'Capricorn/cap16_bx400_track.nc').assign_coords(Source='Capricorn1')

cap2_bx = xr.open_dataset(f_path+'Capricorn/cap18_bx400_track.nc').assign_coords(Source='Capricorn2')

aa_bx = xr.open_dataset(f_path+'AuroraAustralis/aa1718_bx400_track.nc').assign_coords(Source='MARCUS')

tan_bx = xr.open_dataset(f_path+'TAN1802/tan1718_bx400_track.nc').assign_coords(Source='TAN1802')

cgo_bx = xr.open_dataset(f_path+'CGO/cgo1618_bx400_track.nc').assign_coords(Source='CGO')

mqi_bx = xr.open_dataset(f_path+'MQI/mqi1618_bx400_track.nc').assign_coords(Source='MQI')

bx400_data = [rvi_bx,
              r2r_bx,
              i2e_bx,
              cap1_bx,
              cap2_bx,
              aa_bx,
              tan_bx,
              cgo_bx,
              mqi_bx,]

bx400_ds = xr.concat(bx400_data, dim='Source')

In [39]:
bx400_ds.load().to_netcdf(path=output_path+'voyage_bx400.nc')

In [40]:
# load in data
f_path = '/g/data/jk72/ll6859/access_aero_eval/'
rvi_cg = xr.open_dataset(f_path+'RVInvestigator/rvi1619_cg283_track.nc').assign_coords(Source='RVI')

r2r_cg = xr.open_dataset(f_path+'Reef2Rain/r2r16_cg283_track.nc').assign_coords(Source='Reef2Rain')

i2e_cg = xr.open_dataset(f_path+'Ice2Equator/i2e16_cg283_track.nc').assign_coords(Source='Ice2Equ')

cap1_cg = xr.open_dataset(f_path+'Capricorn/cap16_cg283_track.nc').assign_coords(Source='Capricorn1')

cap2_cg = xr.open_dataset(f_path+'Capricorn/cap18_cg283_track.nc').assign_coords(Source='Capricorn2')

aa_cg = xr.open_dataset(f_path+'AuroraAustralis/aa1718_cg283_track.nc').assign_coords(Source='MARCUS')

tan_cg = xr.open_dataset(f_path+'TAN1802/tan1718_cg283_track.nc').assign_coords(Source='TAN1802')

cgo_cg = xr.open_dataset(f_path+'CGO/cgo1618_cg283_track.nc').assign_coords(Source='CGO')

mqi_cg = xr.open_dataset(f_path+'MQI/mqi1618_cg283_track.nc').assign_coords(Source='MQI')

cg283_data = [rvi_cg,
              r2r_cg,
              i2e_cg,
              cap1_cg,
              cap2_cg,
              aa_cg,
              tan_cg,
              cgo_cg,
              mqi_cg,]

cg283_ds = xr.concat(cg283_data, dim='Source')

In [41]:
cg283_ds.load().to_netcdf(path=output_path+'voyage_cg283.nc')